# tiingo 사용법

In [28]:
import keyring
import pandas as pd

In [29]:
from tiingo import TiingoClient

# 설정

In [30]:
api_key = keyring.get_password("tiingo", "noriskfullpush")
config = {}
config['session'] = True
config['api_key'] = api_key
client = TiingoClient(config)

## 테스트 (1) - 모든 ticker 조회

In [9]:
tickers = client.list_stock_tickers()
tickers_df = pd.DataFrame.from_records(tickers)

In [10]:
tickers_df.head()

,ticker,exchange,assetType,priceCurrency,startDate,endDate
0,-P-H,NYSE,Stock,USD,,
1,-P-HIZ,NASDAQ,Stock,USD,2023-08-30,2025-09-29
2,-P-S,NYSE,Stock,USD,2018-08-22,2023-05-05
3,000001,SHE,Stock,CNY,2007-01-04,2025-12-31
4,000002,SHE,Stock,CNY,2007-01-04,2025-12-31


## 테스트 (2) - 거래소, 통화별 종목이 몇개 있는지 확인

In [11]:
tickers_df.groupby(['exchange', 'priceCurrency'])['ticker'].count()

exchange   priceCurrency
           USD               2447
AMEX       USD                 99
ASX        AUD                163
           USD               2111
BATS       USD                 71
CSE        USD                 32
EXPM       USD               3287
LSE        USD                 12
NASDAQ     USD              13397
NMFQS      USD                 36
NYSE       USD               7962
NYSE ARCA  USD                173
NYSE MKT   USD                480
NYSE NAT   USD                  3
OTCBB      USD                643
OTCCE      USD               1094
OTCD       USD                492
OTCGREY    USD               4452
OTCMKTS    USD               1181
OTCQB      USD               1311
OTCQX      USD                813
PINK       USD              15008
SHE        CNY               3386
           HKD                 12
SHEB       HKD                 42
SHG        CNY               2954
           USD                  6
SHGB       USD                 44
Name: ticker, dtype: in

## 테스트 (3) - 종목의 상세정보 조회 (AAPL)

In [12]:
ticker_metadata = client.get_ticker_metadata("AAPL")
print(ticker_metadata)

{'ticker': 'AAPL', 'name': 'Apple Inc', 'description': "Apple Inc. (Apple) designs, manufactures and markets mobile communication and media devices, personal computers, and portable digital music players, and a variety of related software, services, peripherals, networking solutions, and third-party digital content and applications. The Company's products and services include iPhone, iPad, Mac, iPod, Apple TV, a portfolio of consumer and professional software applications, the iOS and OS X operating systems, iCloud, and a variety of accessory, service and support offerings. The Company also delivers digital content and applications through the iTunes Store, App StoreSM, iBookstoreSM, and Mac App Store. The Company distributes its products worldwide through its retail stores, online stores, and direct sales force, as well as through third-party cellular network carriers, wholesalers, retailers, and value-added resellers. In February 2012, the Company acquired app-search engine Chomp.", 

## 테스트 (4) - 종목의 주가 조회

In [13]:
historical_prices = client.get_dataframe("AAPL", startDate='2025-12-26', frequency='daily')
historical_prices.head()

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
date,,,,,,,,,,,,
2025-12-26 00:00:00+00:00,273.40,275.37,272.86,274.160,21521802,273.40,275.37,272.86,274.160,21521802,0.0,1.0
2025-12-29 00:00:00+00:00,273.76,274.36,272.35,272.690,23715213,273.76,274.36,272.35,272.690,23715213,0.0,1.0
2025-12-30 00:00:00+00:00,273.08,274.08,272.28,272.810,22139617,273.08,274.08,272.28,272.810,22139617,0.0,1.0
2025-12-31 00:00:00+00:00,271.86,273.68,271.75,273.060,27293639,271.86,273.68,271.75,273.060,27293639,0.0,1.0
2026-01-02 00:00:00+00:00,271.01,277.84,269.00,272.255,37838054,271.01,277.84,269.00,272.255,37838054,0.0,1.0


## 테스트 (5) - 종목의 Fundamental Data 조회

In [15]:
fundamental_data = client.get_fundamentals_daily("AAPL")
fundamental_data_df = pd.DataFrame.from_records(fundamental_data)
fundamental_data_df.head()

,date,marketCap,enterpriseVal,peRatio,pbRatio,trailingPEG1Y
0,2023-01-03T00:00:00.000Z,1.989628e+12,2.049383e+12,20.905825,35.073745,-2.005059
1,2023-01-04T00:00:00.000Z,2.010150e+12,2.069905e+12,21.121453,35.435503,-2.025739
2,2023-01-05T00:00:00.000Z,1.988833e+12,2.048588e+12,20.897468,35.059723,-2.004257
3,2023-01-06T00:00:00.000Z,2.062010e+12,2.121765e+12,21.666372,36.349715,-2.078002
4,2023-01-09T00:00:00.000Z,2.070442e+12,2.130197e+12,21.754963,36.498344,-2.086499


## 테스트 (6) - 재무제표 다운로드

In [25]:
fundamentals_statements  = client.get_fundamentals_statements("AAPL", startDate='2025-01-01', asReported=True, fmt='csv')

## raw text 데이터를 가공해서 DataFrame 으로
df_fs = pd.DataFrame([x.split(',') for x in fundamentals_statements.split("\n")])

## 첫 번째 행 이름을 열 이름으로 지정한후 삭제
df_fs.columns = df_fs.iloc[0]

## 'date' 열을 인덱스로 지정
df_fs.set_index('date', drop=True, inplace=True)

## 'date' 가 비어있는 부분이 있으므로 제거
df_fs = df_fs[df_fs.index != '']

df_fs.head()

,year,quarter,statementType,dataCode,value
date,,,,,
date,year,quarter,statementType,dataCode,value
2025-10-31,2025,4,incomeStatement,opex,15914000000.0
2025-10-31,2025,4,incomeStatement,ebit,32804000000.0
2025-10-31,2025,4,incomeStatement,sga,7048000000.0
2025-10-31,2025,4,incomeStatement,netIncDiscOps,0.0


## 정리 (1)
티커 목록 조회
```python
## 전체 티커 조회
tickers = client.list_stock_tickers()
tickers_df = pd.DataFrame.from_records(tickers)

## 거래소별, 통화별 종목 몇개인지 확인
tickers_df.groupby(['exchange', 'priceCurrency'])['ticker'].count()
```
<br/>

메타데이터 조회
```python
ticker_metadata = client.get_ticker_metadata("AAPL")
print(ticker_metadata)
```
<br/>

주가 데이터 조회
```python
historical_prices = client.get_dataframe("AAPL", startDate='2025-12-26', frequency='daily')
historical_prices.head()
```
<br/>

종목의 Fundamental Data 조회
```python
fundamental_data = client.get_fundamentals_daily("AAPL")
fundamental_data_df = pd.DataFrame.from_records(fundamental_data)
fundamental_data_df.head()
```

## 테스트 (7) 

In [33]:
from tiingo import TiingoClient
import pandas as pd

# 1. 클라이언트 설정 (API Key 필요)
# config = {'api_key': "YOUR_API_KEY", 'session': True}
# client = TiingoClient(config)

# 2. 데이터 불러오기 (최근 60일치)
ticker = "AAPL"
df = client.get_dataframe(ticker, startDate='2025-11-01')

# 3. 이동평균선 계산
df['MA5'] = df['close'].rolling(window=5).mean()
df['MA20'] = df['close'].rolling(window=20).mean()

# 4. 조건 설정 (5일선이 20일선 위에 있는가?)
df['is_above'] = (df['MA5'] > df['MA20']).astype(int)

# 5. '30일 중 25일 이상' 유지되었는지 확인 (중간 이탈 허용)
# 30일 연속 유지를 원하면 == 30으로 설정
df['condition_met'] = df['is_above'].rolling(window=30).sum() >= 25

# 결과 확인
if df['condition_met'].iloc[-1]:
    print(f"{ticker} 종목이 조건을 만족합니다!")
else:
    print(df['condition_met'])

date
2025-11-03 00:00:00+00:00    False
2025-11-04 00:00:00+00:00    False
2025-11-05 00:00:00+00:00    False
2025-11-06 00:00:00+00:00    False
2025-11-07 00:00:00+00:00    False
2025-11-10 00:00:00+00:00    False
2025-11-11 00:00:00+00:00    False
2025-11-12 00:00:00+00:00    False
2025-11-13 00:00:00+00:00    False
2025-11-14 00:00:00+00:00    False
2025-11-17 00:00:00+00:00    False
2025-11-18 00:00:00+00:00    False
2025-11-19 00:00:00+00:00    False
2025-11-20 00:00:00+00:00    False
2025-11-21 00:00:00+00:00    False
2025-11-24 00:00:00+00:00    False
2025-11-25 00:00:00+00:00    False
2025-11-26 00:00:00+00:00    False
2025-11-28 00:00:00+00:00    False
2025-12-01 00:00:00+00:00    False
2025-12-02 00:00:00+00:00    False
2025-12-03 00:00:00+00:00    False
2025-12-04 00:00:00+00:00    False
2025-12-05 00:00:00+00:00    False
2025-12-08 00:00:00+00:00    False
2025-12-09 00:00:00+00:00    False
2025-12-10 00:00:00+00:00    False
2025-12-11 00:00:00+00:00    False
2025-12-12 00:0